In [1]:
import numpy as np

from Bio import SeqIO

from illumina import run

In [2]:
rng = np.random.default_rng(2024)

# 1. Симуляция секвенирования

In [3]:
genome = ''.join(rng.choice(list("ATGC"), size=50000))
with open("genome.fasta", 'w') as f_out:
    f_out.write(">genome\n")
    f_out.write(genome)
    f_out.write("\n")

In [4]:
ground_truth, reads = run(genome, n_reads=50000, out_filename="reads", rng=rng)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50000/50000 [04:12<00:00, 197.86it/s]


In [5]:
! fastqc reads.fastq

null
Started analysis of reads.fastq
Approx 5% complete for reads.fastq
Approx 10% complete for reads.fastq
Approx 15% complete for reads.fastq
Approx 20% complete for reads.fastq
Approx 25% complete for reads.fastq
Approx 30% complete for reads.fastq
Approx 35% complete for reads.fastq
Approx 40% complete for reads.fastq
Approx 45% complete for reads.fastq
Approx 50% complete for reads.fastq
Approx 55% complete for reads.fastq
Approx 60% complete for reads.fastq
Approx 65% complete for reads.fastq
Approx 70% complete for reads.fastq
Approx 75% complete for reads.fastq
Approx 80% complete for reads.fastq
Approx 85% complete for reads.fastq
Approx 90% complete for reads.fastq
Approx 95% complete for reads.fastq
Approx 100% complete for reads.fastq
Analysis complete for reads.fastq


# 2. Удаление ошибок (Trimmomatic)

In [6]:
! trimmomatic SE -phred33 reads.fastq reads_trimmed.fastq ILLUMINACLIP:TruSeq3-SE:2:30:10 LEADING:3 TRAILING:3 SLIDINGWINDOW:4:15 MINLEN:36

TrimmomaticSE: Started with arguments:
 -phred33 reads.fastq reads_trimmed.fastq ILLUMINACLIP:TruSeq3-SE:2:30:10 LEADING:3 TRAILING:3 SLIDINGWINDOW:4:15 MINLEN:36
Automatically using 1 threads
java.io.FileNotFoundException: /home/nikitos/Study/5th year/2nd_sem/BioinfAlgorithms/BioinformaticsCourse2024/homework/2_1/TruSeq3-SE (No such file or directory)
	at java.base/java.io.FileInputStream.open0(Native Method)
	at java.base/java.io.FileInputStream.open(FileInputStream.java:219)
	at java.base/java.io.FileInputStream.<init>(FileInputStream.java:157)
	at org.usadellab.trimmomatic.fasta.FastaParser.parse(FastaParser.java:54)
	at org.usadellab.trimmomatic.trim.IlluminaClippingTrimmer.loadSequences(IlluminaClippingTrimmer.java:110)
	at org.usadellab.trimmomatic.trim.IlluminaClippingTrimmer.makeIlluminaClippingTrimmer(IlluminaClippingTrimmer.java:71)
	at org.usadellab.trimmomatic.trim.TrimmerFactory.makeTrimmer(TrimmerFactory.java:32)
	at org.usadellab.trimmomatic.Trimmomatic.createTrimmers(T

In [7]:
! fastqc reads_trimmed.fastq

null
Started analysis of reads_trimmed.fastq
Approx 5% complete for reads_trimmed.fastq
Approx 10% complete for reads_trimmed.fastq
Approx 15% complete for reads_trimmed.fastq
Approx 20% complete for reads_trimmed.fastq
Approx 25% complete for reads_trimmed.fastq
Approx 30% complete for reads_trimmed.fastq
Approx 35% complete for reads_trimmed.fastq
Approx 40% complete for reads_trimmed.fastq
Approx 45% complete for reads_trimmed.fastq
Approx 50% complete for reads_trimmed.fastq
Approx 55% complete for reads_trimmed.fastq
Approx 60% complete for reads_trimmed.fastq
Approx 65% complete for reads_trimmed.fastq
Approx 70% complete for reads_trimmed.fastq
Approx 75% complete for reads_trimmed.fastq
Approx 80% complete for reads_trimmed.fastq
Approx 85% complete for reads_trimmed.fastq
Approx 90% complete for reads_trimmed.fastq
Approx 95% complete for reads_trimmed.fastq
Analysis complete for reads_trimmed.fastq


In [8]:
reads_trimmed = {int(rec.id): rec.seq for rec in SeqIO.parse("reads_trimmed.fastq", "fastq")}

In [9]:
trimmomatic_stats = {
    "deleted_reads_total": 0,
    "deleted_reads_with_errors": 0,
    "deleted_reads_no_errors": 0,
    "errors_remove": 0,
    "read_with_errors_left": 0,
}

for i in range(50_000):
    read_seq = reads[i][1]
    _, start, read_length = ground_truth[i]
    if i not in reads_trimmed:
        trimmomatic_stats["deleted_reads_total"] += 1
        if genome[start: start + read_length] == read_seq:
            trimmomatic_stats["deleted_reads_no_errors"] += 1
        else:
            trimmomatic_stats["deleted_reads_with_errors"] += 1
    else:
        read_seq_trimmed = str(reads_trimmed[i])
        shift = read_seq.find(read_seq_trimmed)
        if genome[start + shift: start + shift + len(read_seq_trimmed)] == read_seq_trimmed:
            trimmomatic_stats["errors_remove"] += 1
        else:
            trimmomatic_stats["read_with_errors_left"] += 1

In [10]:
for k, v in trimmomatic_stats.items():
    print(k, ':', v)

deleted_reads_total : 1352
deleted_reads_with_errors : 1352
deleted_reads_no_errors : 0
errors_remove : 84
read_with_errors_left : 48564


# 3. Коррекция ошибок

In [11]:
! lighter

Usage: ./lighter [OPTIONS]
OPTIONS:
Required parameters:
	-r seq_file: seq_file is the path to the sequence file. Can use multiple -r to specifiy multiple sequence files
	             The file can be fasta and fastq, and can be gzip'ed with extension *.gz.
	             When the input file is *.gz, the corresponding output file will also be gzip'ed.
	-k kmer_length genome_size alpha: (see README for information on setting alpha)
					or
	-K kmer_length genom_size: in this case, the genome size should be relative accurate.
Other parameters:
	-od output_file_directory: (default: ./)
	-t num_of_threads: number of threads to use (default: 1)
	-maxcor INT: the maximum number of corrections within a 20bp window (default: 4)
	-trim: allow trimming (default: false)
	-discard: discard unfixable reads. Will LOSE paired-end matching when discarding (default: false)
	-noQual: ignore the quality socre (default: false)
	-newQual ascii_quality_score: set the quality for the bases corrected to the spe

In [12]:
k_mer_lengths = [10, 15, 20, 25, 30]

for k in k_mer_lengths:
    ! lighter -r reads.fastq -K {k} 50000 -od "kmer{k}"

[2024-06-14 17:12:31] =============Start====================
[2024-06-14 17:12:31] Scanning the input files to infer alpha(sampling rate)
[2024-06-14 17:12:31] Average coverage is 249.524 and alpha is 0.028
[2024-06-14 17:12:31] Bad quality threshold is "$"
[2024-06-14 17:12:31] Finish sampling kmers
[2024-06-14 17:12:31] Bloom filter A's false positive rate: 0.166457
[2024-06-14 17:12:31] The error rate is high. Lighter adjusts -maxcor to 5 and bad quality threshold to "%".
[2024-06-14 17:12:32] Finish storing trusted kmers
[2024-06-14 17:12:32] Finish error correction
Processed 50000 reads:
	3 are error-free
	Corrected 512017 bases(10.240954 corrections for reads with errors)
	Trimmed 0 reads with average trimmed bases 0.000000
	Discard 0 reads
[2024-06-14 17:12:33] =============Start====================
[2024-06-14 17:12:33] Scanning the input files to infer alpha(sampling rate)
[2024-06-14 17:12:33] Average coverage is 249.524 and alpha is 0.028
[2024-06-14 17:12:33] Bad quality th

In [13]:
def calc_metrics(reads_cor, reads, ground_truth):
    tp = tn = fp = fn = 0
    miscorrection = 0

    for i in range(50_000):
        _, start, read_length = ground_truth[i]
        for c1, c2, c3 in zip(reads_cor[i], reads[i][1], genome[start: start + read_length]):
            if c1 == c2 == c3:
                tn += 1
            elif c1 != c2 == c3:
                fp += 1
            elif c1 == c2 != c3:
                fn += 1
            elif c1 == c3 != c2:
                tp += 1
            else:
                miscorrection += 1

    print(f"TP: {tp}")
    print(f"FP: {fp}")
    print(f"TN: {tn}")
    print(f"FN: {fn}")
    print(f"Miscorrections: {miscorrection}")
    print(f"Precision: {tp / (tp + fp)}")
    print(f"Recall: {tp / (tp + fn)}")

In [14]:
for k in k_mer_lengths:
    reads_cor = {int(rec.id): rec.seq for rec in SeqIO.parse(f"kmer{k}/reads.cor.fq", "fastq")}
    print(f"k-mer length: {k}")
    calc_metrics(reads_cor, reads, ground_truth)
    print('-' * 20)

k-mer length: 10
TP: 453806
FP: 54200
TN: 11826530
FN: 137673
Miscorrections: 4011
Precision: 0.89330834675181
Recall: 0.7672394117119965
--------------------
k-mer length: 15
TP: 528906
FP: 33
TN: 11880697
FN: 66583
Miscorrections: 1
Precision: 0.999937610953248
Recall: 0.8881876911244372
--------------------
k-mer length: 20
TP: 525909
FP: 11
TN: 11880719
FN: 69580
Miscorrections: 1
Precision: 0.999979084271372
Recall: 0.8831548525665461
--------------------
k-mer length: 25
TP: 332164
FP: 43
TN: 11880687
FN: 263324
Miscorrections: 2
Precision: 0.9998705626311306
Recall: 0.5578013326884841
--------------------
k-mer length: 30
TP: 103847
FP: 52
TN: 11880678
FN: 491640
Miscorrections: 3
Precision: 0.9994995139510486
Recall: 0.1743900370621021
--------------------


In [15]:
! fastqc kmer20/reads.cor.fq

null
Started analysis of reads.cor.fq
Approx 5% complete for reads.cor.fq
Approx 10% complete for reads.cor.fq
Approx 15% complete for reads.cor.fq
Approx 20% complete for reads.cor.fq
Approx 25% complete for reads.cor.fq
Approx 30% complete for reads.cor.fq
Approx 35% complete for reads.cor.fq
Approx 40% complete for reads.cor.fq
Approx 45% complete for reads.cor.fq
Approx 50% complete for reads.cor.fq
Approx 55% complete for reads.cor.fq
Approx 60% complete for reads.cor.fq
Approx 65% complete for reads.cor.fq
Approx 70% complete for reads.cor.fq
Approx 75% complete for reads.cor.fq
Approx 80% complete for reads.cor.fq
Approx 85% complete for reads.cor.fq
Approx 90% complete for reads.cor.fq
Approx 95% complete for reads.cor.fq
Approx 100% complete for reads.cor.fq
Analysis complete for reads.cor.fq


# 4. Сделайте какой-то вывод

По результатам мы видим, что редактирование ошибок предпочтительнее, чем удаление ридов с плохим качеством. При оптимальном выборе настроек алгоритма исправления ошибок можно добиться высокой точности и полноты исправления ошибок. Кроме этого, при исправлении ошибок не теряются данные в отличии от trimmomatic